In [1]:
import pandas as pd


In [42]:
training_features = pd.read_csv('training_set_features.csv')
training_labels = pd.read_csv('training_set_labels.csv')
training_features.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [40]:
# clean data
# do not drop lines but dropping columns is ok


def cleanData(df : pd.DataFrame):
    # drop employment_industry and employment_occupation because nearly 50% NaNs and filled with dummy-text
    df.drop(['employment_industry', 'employment_occupation'], axis = 1, inplace=True)
    
    ## care for NaNs
    # fill NaN in health-insurance, doctor_recc_h1n1, doctor_recc_seasonal with value 0
    df[['health_insurance','doctor_recc_h1n1','doctor_recc_seasonal']]=df[['health_insurance','doctor_recc_h1n1','doctor_recc_seasonal']].fillna(0)
    # fill NaN in income-poverty with 'missing'
    df['income_poverty']=df['income_poverty'].fillna('missing')
    # fill NaN in rent_or_own with own
    df['rent_or_own']=df['rent_or_own'].fillna('own')
    #fill NaN in employment_status with 'missing'
    df['employment_status']=df['employment_status'].fillna('missing')
    #fill NaN in marital_status with 'missing'
    df['marital_status']=df['marital_status'].fillna('missing')
    #fill NaN in education with 'missing'
    df['education']=df['education'].fillna('College Graduate')
    #fill NaN in household_children and household_adults with 'missing'
    df[['household_children', 'household_adults']]=df[['household_children', 'household_adults']].fillna('missing')
    #fil NaN in opinion_seas_sick_from_vacc, opinion_seas_risk and opinion_seas_vacc_effective with missing
    df[['opinion_seas_sick_from_vacc', 'opinion_seas_risk', 'opinion_seas_vacc_effective']] = df[['opinion_seas_sick_from_vacc','opinion_seas_risk', 'opinion_seas_vacc_effective']].fillna('missing')
    #fil NaN in opinion_h1n1_sick_from_vacc, oopinion_h1n1_risk and opinion_seas_vacc_effective with missing
    df[['opinion_h1n1_sick_from_vacc', 'opinion_h1n1_risk', 'opinion_h1n1_vacc_effective']] = df[['opinion_h1n1_sick_from_vacc', 'opinion_h1n1_risk', 'opinion_h1n1_vacc_effective']].fillna('missing')
    #fill NaN in health_worker with 0
    df['health_worker']=df['health_worker'].fillna(0)
    #fill NaN in child_under_6_months with 0
    df['child_under_6_months']=df['child_under_6_months'].fillna(0)
    #fill NaN in chronic_med_condition with 'missing'
    df['chronic_med_condition']=df['chronic_med_condition'].fillna('missing')
    #fill NaN in behavorial flags with 'missing'
    df[['behavioral_antiviral_meds','behavioral_avoidance','behavioral_face_mask','behavioral_wash_hands','behavioral_large_gatherings','behavioral_outside_home','behavioral_touch_face']]=df[['behavioral_antiviral_meds','behavioral_avoidance','behavioral_face_mask','behavioral_wash_hands','behavioral_large_gatherings','behavioral_outside_home','behavioral_touch_face']].fillna('missing')
    #fill NaN in h1ns_knowledge with 'missing'
    df['h1n1_knowledge']=df['h1n1_knowledge'].fillna('missing')
    #fill NaN in h1ns_knowledge with 'missing'
    df['h1n1_knowledge']=df['h1n1_knowledge'].fillna('missing')
    #fill NaN in h1ns_concern with 'missing'
    df['h1n1_concern']=df['h1n1_concern'].fillna('missing')
    
    #set correct data-types
    df = df.astype('category')
    
    #ToDo: handle categorical-data
    
    return df

trainingData = pd.merge(training_features, training_labels, on= 'respondent_id')
cleaned_trainingData = cleanData(trainingData)
cleaned_trainingData.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,seasonal_vaccine
0,0,1,0,0,0,0,0,0,1,1,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0,0,0,0
1,1,3,2,0,1,0,1,0,1,1,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0,0,0,1
2,2,1,1,0,1,0,0,0,0,0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2,0,0,0
3,3,1,1,0,1,0,1,1,0,0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0,0,0,1
4,4,2,1,0,1,0,1,1,0,1,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1,0,0,0


In [ ]:
def findUnfilledCollumns(df : pd.DataFrame):
    print(df.isnull().sum(axis = 0))
    return

findUnfilledCollumns(cleaned_trainingData)

In [34]:
training_features.describe()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26707.000000,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,...,25903.000000,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000
mean,13353.000000,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,...,0.111918,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583
std,7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,...,0.315271,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,6676.500000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,0.000000,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,13353.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,...,0.000000,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,20029.500000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,0.000000,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000
